## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [2]:
#calculate the sample size of each group
n_ctrl = train_data.groupby('Promotion')['Promotion'].count()[0]
n_treat = train_data.groupby('Promotion')['Promotion'].count()[1]
n_treat

42364

In [3]:
#calculate the number of purchaser in each group
pur_ctrl = train_data.groupby('Promotion')['purchase'].sum()[0]
pur_treat = train_data.groupby('Promotion')['purchase'].sum()[1]
pur_treat

721

In [4]:
#calculate the purchase rate of each group
count = np.array([pur_ctrl, pur_treat])
obs = np.array([n_ctrl, n_treat])
count

array([319, 721])

In [5]:
# z_test if the IRR is significant
import statsmodels.stats.proportion as propstest
z, p = propstest.proportions_ztest(count, obs)

In [6]:
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [7]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()

In [8]:
# generate feature matrix x and reponse y
pur = train_data[train_data['purchase'] == 1]
not_pur = train_data[train_data['purchase'] == 0].sample(1040)
sample = pd.concat([pur, not_pur], axis = 0)

y = sample['purchase']

cat_var = ['V1', 'V4', 'V5', 'V6', 'V7']
for var in cat_var:
    sample = pd.concat([sample.drop(var, axis = 1), pd.get_dummies(sample[var], prefix = var)], axis = 1)

x = sample.loc[:,'V2':]

x.head()

,V2,V3,V1_0,V1_1,V1_2,V1_3,V4_1,V4_2,V5_1,V5_2,V5_3,V5_4,V6_1,V6_2,V6_3,V6_4,V7_1,V7_2
107,33.030562,-0.732194,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0
328,31.760033,1.518828,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1
352,17.512709,1.259095,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1
383,31.043735,-1.338239,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
423,34.901993,-0.905350,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1


In [9]:
# use x and y to train the logistic model
lgr.fit(x, y)
lgr.predict(x).sum()

1123

In [10]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    x = df
    
    for var in cat_var:
        x = pd.concat([x.drop(var, axis = 1), pd.get_dummies(x[var], prefix = var)], axis = 1)
        
    
    #x = x.loc[:, 'V3':]
    promotion = lgr.predict(x)
    
    my_map = {0: "No", 1: "Yes"}
    promotion = np.array([my_map[i] for i in promotion])
    
    return promotion

In [11]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0184.

Your nir with this strategy is 317.30.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.018391885020071913, 317.29999999999995)